## Learning Objectives

- Learn all of the methods in pandas for data-frame manipulation
- The dataset we use is Titanic dataset
- Apply visualization to data-frame

### Lets make Pandas dataframe from titanic csv file 

In [1]:
import numpy as np 
from pyspark import SparkContext

sc = SparkContext()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
df = spark.read.csv('titanic.csv', header=True, inferSchema=True)

### Lets look at the first 5 rows of dataframe

In [4]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [5]:
print("Shape:", (df.count(), len(df.columns)))

Shape: (891, 12)


### Titanic Dataset Description

VARIABLE DESCRIPTIONS:  
survival        Survival  
                (0 = No; 1 = Yes)  
pclass          Passenger Class  
                (1 = 1st; 2 = 2nd; 3 = 3rd)  
name            Name  
sex             Sex  
age             Age  
sibsp           Number of Siblings/Spouses Aboard  
parch           Number of Parents/Children Aboard  
ticket          Ticket Number  
fare            Passenger Fare  
cabin           Cabin  
embarked        Port of Embarkation  
                (C = Cherbourg; Q = Queenstown; S = Southampton)  

### Plot how many of the passengers were children, youth, middle age and old?

In [6]:
import matplotlib.pyplot as plt
from seaborn import distplot


df.groupby("Age").count().show()

+----+-----+
| Age|count|
+----+-----+
| 8.0|    4|
|70.0|    2|
| 7.0|    3|
|20.5|    1|
|49.0|    6|
|29.0|   20|
|40.5|    2|
|64.0|    2|
|47.0|    9|
|42.0|   13|
|24.5|    1|
|44.0|    9|
|35.0|   18|
|null|  177|
|62.0|    4|
|18.0|   26|
|80.0|    1|
|34.5|    1|
|39.0|   14|
| 1.0|    7|
+----+-----+
only showing top 20 rows



### How many of Age values are empty (or null)?

In [7]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



### Create a new column as gender, when Sex is female it is zero when sex is male it is one

In [8]:
df = df.withColumn("Gender", when(df['Sex'] == "male", 1).otherwise(0))
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     1|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|     0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|     0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|     0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|     1|
+-----------+--------+--

### We have one more column (check it)

In [9]:
(df.count(), len(df.columns))

(891, 13)

### Show the majority of Age range

In [10]:
df.groupby("Age").count().sort('count', ascending=False).show(5)

+----+-----+
| Age|count|
+----+-----+
|null|  177|
|24.0|   30|
|22.0|   27|
|18.0|   26|
|30.0|   25|
+----+-----+
only showing top 5 rows



### List all of the Ages that are not null

In [11]:
df.where(col("Age").isNotNull()).show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     1|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|     0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|     0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|     0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|     1|
+-----------+--------+--

### Slice the dataframe for those whose Embarked section was 'C'

In [12]:
c_embark_df = df.filter(df['Embarked'] == "C")
c_embark_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|PC 17599|71.2833|  C85|       C|     0|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|  237736|30.0708| null|       C|     0|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|    2649|  7.225| null|       C|     0|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|    2631|  7.225| null|       C|     1|
|         31|       0|     1|Uruchurtu, Don. M...|  male|40.0|    0|    0|PC 17601|27.7208| null|       C|     1|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-----

### Plot the Age range for those whose Embraked were 'C'

In [13]:
c_embark_df.groupby("Age").count().sort('count', ascending=False).show(5)

+----+-----+
| Age|count|
+----+-----+
|null|   38|
|30.0|    7|
|24.0|    7|
|17.0|    5|
|22.0|    5|
+----+-----+
only showing top 5 rows



### Describe a specific column 

In [14]:
df.describe(['Embarked']).show()

+-------+--------+
|summary|Embarked|
+-------+--------+
|  count|     889|
|   mean|    null|
| stddev|    null|
|    min|       C|
|    max|       S|
+-------+--------+



### How many unique values does the 'Embraked' have?

In [15]:
df.select('Embarked').distinct().dropna().count()

3

### Count the different 'Embarked' values the dataframe has

In [16]:
df.groupby('Embarked').count().show(4)

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



### Count the different 'Embarked' values the dataframe has and plot horizontaly

In [17]:
# Done above (but not horizontally)

### Another way to do the count and plot it

In [18]:
# same as the last two

In [19]:
df.groupby('Embarked').count()

DataFrame[Embarked: string, count: bigint]

In [20]:
df.groupby('Sex').count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



### Plot how many of the passengers were children, youth, middle age and old based on there Sex for those who 'Embarked' in section 'C'?

In [23]:
# df['Sex'].value_counts().plot(kind='bar')
# Skip

In [24]:
# df['Sex'].value_counts().plot(kind='pie')
# Skip

In [37]:
df.filter(df['Embarked'] == 'C').groupby('Sex').agg({'Age':'mean'}).show()

+------+-----------------+
|   Sex|         avg(Age)|
+------+-----------------+
|female|28.34426229508197|
|  male|32.99884057971015|
+------+-----------------+



### Show difference in counts of Female vs Male ages:

In [49]:
print('Female ages and counts:')
df.filter((df['Embarked'] == 'C') & (df['Sex'] == 'female')).groupby('Age').count().sort('Age').show(5)

Female ages and counts:
+----+-----+
| Age|count|
+----+-----+
|null|   12|
|0.75|    2|
| 1.0|    1|
| 3.0|    1|
| 4.0|    1|
+----+-----+
only showing top 5 rows



In [48]:
print('Male ages and counts:')
df.filter((df['Embarked'] == 'C') & (df['Sex'] == 'male')).groupby('Age').count().sort('Age').show(5)

Male ages and counts:
+----+-----+
| Age|count|
+----+-----+
|null|   26|
|0.42|    1|
| 1.0|    1|
|11.0|    1|
|12.0|    1|
+----+-----+
only showing top 5 rows



### What is the average Age for female and male (based on sex) for those who have 'Embarked' on section 'C'?

In [58]:
df.filter(df['Embarked'] == 'C').groupby('Sex').agg({'Age': 'mean'}).show()

+------+-----------------+
|   Sex|         avg(Age)|
+------+-----------------+
|female|28.34426229508197|
|  male|32.99884057971015|
+------+-----------------+



### Which Age is the oldest for female and male (based on sex) for those who have 'Embarked' on section 'C'?

In [59]:
df.filter(df['Embarked'] == 'C').groupby('Sex').agg({'Age': 'max'}).show()

+------+--------+
|   Sex|max(Age)|
+------+--------+
|female|    60.0|
|  male|    71.0|
+------+--------+



### For different Ages, plot the Fare they have paid?

In [ ]:
# Skip plot

### Plot how percentage Survived for two Sex group based on the passengers class 

In [74]:
print('Male Pclass Survival percentages:')
df.filter(df['Sex'] == 'male').groupby('Pclass').agg({'Survived': 'avg'}).sort('Pclass').show()
print('Female Pclass Survival percentages:')
df.filter(df['Sex'] == 'female').groupby('Pclass').agg({'Survived': 'avg'}).sort('Pclass').show()

Male Pclass Survival percentages:
+------+-------------------+
|Pclass|      avg(Survived)|
+------+-------------------+
|     1|0.36885245901639346|
|     2| 0.1574074074074074|
|     3|0.13544668587896252|
+------+-------------------+

Female Pclass Survival percentages:
+------+------------------+
|Pclass|     avg(Survived)|
+------+------------------+
|     1|0.9680851063829787|
|     2|0.9210526315789473|
|     3|               0.5|
+------+------------------+



### Show how many male or female were in different Passenger classes

In [71]:
print('Male Pclass counts:')
df.filter(df['Sex'] == 'male').groupby('Pclass').count().show()
print('Female Pclass counts:')
df.filter(df['Sex'] == 'female').groupby('Pclass').count().show()

Male Pclass counts:
+------+-----+
|Pclass|count|
+------+-----+
|     1|  122|
|     3|  347|
|     2|  108|
+------+-----+

Female Pclass counts:
+------+-----+
|Pclass|count|
+------+-----+
|     1|   94|
|     3|  144|
|     2|   76|
+------+-----+



### Stack plot of count based on Sex and Survival for different Passenger Class

In [75]:
# Skip plot

### Sometimes it is hard to read values from plot, what are the number of female and male at each Passenger Class

In [79]:
# Done two cells above

### How to represent the above cross tab in percentage and graphically present 

In [80]:
# skip heatmap plot

## Question:

What percent of passengers embarked at C?

In [95]:
ans = round(df.filter(df['Embarked'] == 'C').count() / df.count(), 4) * 100
print(f'{ans}% of passengers embarked at C.')

18.86% of passengers embarked at C.


What percent of female passengers embarked at C?

In [96]:
female_df = df.filter(df['Sex'] == 'female')

ans = round(female_df.filter(female_df['Embarked'] == 'C').count() / female_df.count(), 4) * 100
print(f'{ans}% of females embarked at C.')

23.25% of females embarked at C.


This question is different from above:
What percent of passengers embarked at C were female?

In [100]:
embarked_c_df = df.filter(df['Embarked'] == 'C')

ans = round(embarked_c_df.filter(embarked_c_df['Sex'] == 'female').count() / embarked_c_df.count(), 3) * 100
print(f'{ans}% of passengers that embarked at C were female.')

43.5% of passengers that embarked at C were female.
